# Implement XLSM template

In [1]:
import warnings
from collections import defaultdict
from pathlib import Path

from planetary_coverage.events import EventsDict, EventsList
from planetary_coverage.html import table
from planetary_coverage import datetime

from openpyxl import load_workbook

from majis.itl.reader import read_itl
from majis.misc import fmt_datetime


In [4]:
with warnings.catch_warnings():
    # Disable openpyxl warning about unsupported extension ()
    warnings.simplefilter('ignore')

    template = load_workbook('timeline.xltm', keep_vba=True)

template

In [ ]:
template.save('foo.xlsm')

In [ ]:
template.sheetnames

['Timeline', 'Color code', 'info', 'change log', 'internal']

## Change log

In [ ]:
rows = list(template['change log'])
row = rows[1]
cell = row[3]
cell_empty = row[4]

cell, cell_empty

(<ReadOnlyCell 'change log'.D2>, <EmptyCell>)

In [ ]:
class TimelineChangeLog:
    """MAJIS timeline template change log."""

    sheet_name = 'change log'

    def __init__(self, template):
        self.log = self.read_changelog(list(template[self.sheet_name])[1:])

    def __str__(self):
        return '\n\n'.join(
            f'>>> {version:>6} | {date} | {author}\n{changes}'
            for version, date, author, changes in self.log
        )

    def __getitem__(self, i):
        return self.log[i]

    def __iter__(self):
        return iter(self.log)

    def _repr_html_(self):
        return table(self.log, header=['Version', 'Date', 'Author', 'Changes'])

    def read_changelog(self, rows) -> (list, list):
        """Read changelog changes grouped by version, date and author."""
        log = defaultdict(list)

        # Group release changes
        key = None, None, None
        for row in rows:
            version, date, change, author = (cell.value for cell in row[:4])

            if change and change != '…':
                key = (
                    str(version) if version else key[0],
                    str(date.date()) if date else key[1],
                    str(author) if author else key[2],
                )

                log[key].extend(change.splitlines())

        # Reverse changes order (latest first) and add line returns between changes
        return [(*key, '\n'.join(log[key])) for key in reversed(log)]


change_log = TimelineChangeLog(template)

change_log.log

[('1.9.10', '2024-05-29', 'Cydalise Dumesnil', 'Add new column "Comments"'),
 ('1.9.9',
  '2024-05-23',
  'Vincent Carlier',
  'Add new columns "VI Gain", "VI Offset", "IR Gain", "IR Offset"'),
 ('1.9.8',
  '2024-05-13',
  'Vincent Carlier',
  "Fix issues related to the decimal separator ',' or '.' depending on the host PC :\n- new variable name 'decimal_separator' used to compute the time 'wrt C/A' from 'UTC' in the cells\n- fix of the macro DIFF_TIME_ms used to compute the DV in case of continuous acquisition\nAdd a note about 'scanner_timetot' computation\nAdd a note about decimal separator in the time 'wrt C/A' columns"),
 ('1.9.7', '2024-04-25', 'Vincent Carlier', 'Add spectral masks'),
 ('1.9.7',
  '2024-03-21',
  'Vincent Carlier',
  'Change the names BROWSE_TABLE into BROWSE_PARAMETERS\nChange column names: "Browse" into "Browse Parameters"\nFix fomulas for nomi and brow to take into account VI_Flag and IR_Flag\nAdd bullet tips in 2nd row (copy of footer comments)'),
 ('1.9.7',

In [ ]:
change_log

Version,Date,Author,Changes
1.9.10,2024-05-29,Cydalise Dumesnil,"Add new column ""Comments"""
1.9.9,2024-05-23,Vincent Carlier,"Add new columns ""VI Gain"", ""VI Offset"", ""IR Gain"", ""IR Offset"""
1.9.8,2024-05-13,Vincent Carlier,"Fix issues related to the decimal separator ',' or '.' depending on the host PC : - new variable name 'decimal_separator' used to compute the time 'wrt C/A' from 'UTC' in the cells - fix of the macro DIFF_TIME_ms used to compute the DV in case of continuous acquisition Add a note about 'scanner_timetot' computation Add a note about decimal separator in the time 'wrt C/A' columns"
1.9.7,2024-04-25,Vincent Carlier,Add spectral masks
1.9.7,2024-03-21,Vincent Carlier,"Change the names BROWSE_TABLE into BROWSE_PARAMETERS Change column names: ""Browse"" into ""Browse Parameters"" Fix fomulas for nomi and brow to take into account VI_Flag and IR_Flag Add bullet tips in 2nd row (copy of footer comments)"
1.9.7,2024-03-14,Vincent Carlier,Split de number of spectels from the spectral table into VI and IR. Split de number of spectels from the browse parameters into VI and IR.
1.9.6,2024-03-14,Vincent Carlier,"Add a macro which resets the unused values if Mirror Flag is ""DISABLE"" Add a data validation on start_angle and stop_angle (from -2.1 to +2.1)"
1.9.6,2024-03-13,Vincent Carlier,- reorganisation of the order of columns
1.9.6,2024-03-11,Vincent Carlier,"- Change column names: ""Observation ID"" changed in ""OBS_NAME"""
1.9.6,2024-03-04,Vincent Carlier,- Add a check in DV cell to check that the number of CU Frame entered by hand is equal to the computed value - fix the DIFF_TIME_CA_ms function to work with column name instead of cell name - Update color code - Add comments


In [ ]:
print(change_log)

>>> 1.9.10 | 2024-05-29 | Cydalise Dumesnil
Add new column "Comments"

>>>  1.9.9 | 2024-05-23 | Vincent Carlier
Add new columns "VI Gain", "VI Offset", "IR Gain", "IR Offset"

>>>  1.9.8 | 2024-05-13 | Vincent Carlier
Fix issues related to the decimal separator ',' or '.' depending on the host PC :
- new variable name 'decimal_separator' used to compute the time 'wrt C/A' from 'UTC' in the cells
- fix of the macro DIFF_TIME_ms used to compute the DV in case of continuous acquisition
Add a note about 'scanner_timetot' computation
Add a note about decimal separator in the time 'wrt C/A' columns

>>>  1.9.7 | 2024-04-25 | Vincent Carlier
Add spectral masks

>>>  1.9.7 | 2024-03-21 | Vincent Carlier
Change the names BROWSE_TABLE into BROWSE_PARAMETERS
Change column names: "Browse" into "Browse Parameters"
Fix fomulas for nomi and brow to take into account VI_Flag and IR_Flag
Add bullet tips in 2nd row (copy of footer comments)

>>>  1.9.7 | 2024-03-14 | Vincent Carlier
Split de number of 

## Timeline

In [ ]:
timeline = template['Timeline']

timeline

<Worksheet "Timeline">

In [ ]:
fields = [cell.value for cell in next(timeline.rows)]

fields

['Event Name',
 'Phase',
 'block',
 'Comments',
 'OBS_NAME',
 'start_time_relative_ca',
 'stop_time_relative_ca',
 'start_time_utc',
 'stop_time_utc',
 'pointing desc',
 'MAJIS resol',
 'prime',
 'MAJIS ODF name',
 'ITL name',
 'Mirror Flag',
 'start_angle',
 'start_scan_speed',
 'stop_scan_speed',
 'Scanner step per frame',
 'stop_angle',
 'scanner_timetot (ms)',
 'First CU_frame start wrt C/A',
 'Last CU_frame stop wrt C/A',
 'First CU_frame start (UTC)',
 'Last CU_frame stop (UTC)',
 'cu_trep_ms',
 'spatial_binning',
 'obs durat (sec) (w/o borders)',
 'nb_cu_frames_tot',
 'nb_cu_frames_tot (computed)',
 'ppe',
 'Spectral Mapping',
 'Number of spectels after spectral binning',
 'Nb of bits per spectel after compression',
 'datarate (bits/s)',
 'DV (Mbits)',
 'Browse Parameters',
 'Number of browse spectels',
 'Nb of bits per browse spectel after compression',
 'browse datarate (bits/s)',
 'browse DV (Mbits)',
 'Image Mode',
 'Dark Strategy',
 'Dark Subtraction',
 'VI flag',
 'IR flag

In [ ]:
BINNING = {
    '1': 'No Binning',
    '2': 'Binning x2',
    '4': 'Binning x4',
}

MAPPING = {
    # ITL parameters
    'OBS_NAME': ('OBS_NAME', str),
    'start_angle': ('START_ANGLE', float),                          # °
    'start_scan_speed': ('START_SCAN_SPEED', float),                # °/s
    'stop_scan_speed': ('STOP_SCAN_SPEED', float),                  # °/s
    'Scanner step per frame': ('SYNCHRONOUS', float),               # (-3, 0, 3)
    'stop_angle': ('STOP_ANGLE', float),                            # °
    'First CU_frame start (UTC)': ('t_start', fmt_datetime),        # YYYY-MM-DDThh:mm:ss.ms
    'Last CU_frame stop (UTC)': ('t_end', fmt_datetime),            # YYYY-MM-DDThh:mm:ss.ms
    'cu_trep_ms': ('CU_TREP', lambda s: int(s.replace('ms', ''))),  # ms
    'spatial_binning': ('BINNING', lambda i: BINNING[i]),
    'nb_cu_frames_tot': ('CU_FRAME', int),
    'ppe': ('PPE', int),
    'Start Row VI': ('START_ROW_VIS', int),
    # Other
    'Comments': ('COMMENTS', str),
    'prime': ('PRIME', lambda b: 'MAJIS' if b else 'other'),
}

    # 'ITL name': ('', str),
    # 'Mirror Flag': ('', str),  # 'ENABLE' if 'START_SCAN_SPEED' or 'STOP_SCAN_SPEED' else DISABLE
    # 'First CU_frame start wrt C/A': ('t_start', fmt_td(dt - dt_ca)),
    # 'Last CU_frame stop wrt C/A': ('t_end', fmt_td(dt- dt_ca)),


In [ ]:
cols = [col for col in next(timeline.columns) if col.value]

cols

[<Cell 'Timeline'.A1>, <Cell 'Timeline'.A2>]

In [ ]:
col = cols[1]

col

<Cell 'Timeline'.A2>

In [ ]:
col.coordinate

'A2'

In [ ]:
timeline['B']

(<Cell 'Timeline'.B1>,
 <Cell 'Timeline'.B2>,
 <Cell 'Timeline'.B3>,
 <Cell 'Timeline'.B4>,
 <Cell 'Timeline'.B5>,
 <Cell 'Timeline'.B6>,
 <Cell 'Timeline'.B7>,
 <Cell 'Timeline'.B8>,
 <Cell 'Timeline'.B9>,
 <Cell 'Timeline'.B10>,
 <Cell 'Timeline'.B11>,
 <Cell 'Timeline'.B12>,
 <Cell 'Timeline'.B13>,
 <Cell 'Timeline'.B14>,
 <Cell 'Timeline'.B15>,
 <Cell 'Timeline'.B16>,
 <Cell 'Timeline'.B17>,
 <Cell 'Timeline'.B18>,
 <Cell 'Timeline'.B19>,
 <Cell 'Timeline'.B20>,
 <Cell 'Timeline'.B21>,
 <Cell 'Timeline'.B22>,
 <Cell 'Timeline'.B23>,
 <Cell 'Timeline'.B24>,
 <Cell 'Timeline'.B25>,
 <Cell 'Timeline'.B26>,
 <Cell 'Timeline'.B27>,
 <Cell 'Timeline'.B28>,
 <Cell 'Timeline'.B29>)

In [ ]:
col.column_letter

'A'

In [ ]:
col.col_idx

1

In [ ]:
itl = read_itl('../../tests/data/absolute_time.itl', flat=True)

itl

,t_start,t_end,INSTRUMENT,SCENARIO,OBS_NAME,TARGET,CU_TREP,CU_FRAME,BINNING,PPE,START_ROW_VIS,START_ANGLE,STOP_ANGLE,SYNCHRONOUS,START_SCAN_SPEED,STOP_SCAN_SPEED,PRIME,COMMENTS
0,2024-08-19T21:18:09.850,2024-08-19T21:18:59.350,MAJIS,XXXX_XX,MAJ_LEGA_SCAN_001,MOON,500ms,99,1,400,100,+0.00000,+0.00000,0,+0.0000000000,+0.0000000000,True,CROSSING OF THE MOON'S TERMINATOR / MULTI LINES
1,2024-08-20T21:25,2024-08-20T21:27:54,MAJIS,XXXX_XX,MAJ_EGA_SCAN_001,EARTH,200ms,870,1,128,372,-1.99951,+2.09682,0,+0.0235421317,+0.0235421317,True,NIGHT OBSERVATION OF KUALA LUMPUR AND SUMATRA ISLAND EAST COAST


In [ ]:
DEFAULT_TEMPLATE = Path('timeline.xltm')

BINNING = {
    '1': 'No Binning',
    '2': 'Binning x2',
    '4': 'Binning x4',
}

ITL_MAPPING = {
    # Timeline field: (ITL key, value formatter)
    'OBS_NAME': ('OBS_NAME', str),
    'start_angle': ('START_ANGLE', float),                          # °
    'start_scan_speed': ('START_SCAN_SPEED', float),                # °/s
    'stop_scan_speed': ('STOP_SCAN_SPEED', float),                  # °/s
    'Scanner step per frame': ('SYNCHRONOUS', float),               # (-3, 0, 3)
    'stop_angle': ('STOP_ANGLE', float),                            # °
    'First CU_frame start (UTC)': ('t_start', fmt_datetime),        # YYYY-MM-DDThh:mm:ss.ms
    'Last CU_frame stop (UTC)': ('t_end', fmt_datetime),            # YYYY-MM-DDThh:mm:ss.ms
    'cu_trep_ms': ('CU_TREP', lambda s: int(s.replace('ms', ''))),  # ms
    'spatial_binning': ('BINNING', lambda i: BINNING[i]),
    'nb_cu_frames_tot': ('CU_FRAME', int),
    'ppe': ('PPE', int),
    'Start Row VI': ('START_ROW_VIS', int),
    'prime': ('PRIME', lambda b: 'MAJIS' if b else 'other'),
    'Comments': ('COMMENTS', str),
}


class Timeline:
    """MAJIS timeline from template.

    You can either append an existing timeline template
    or use the default one.

    Warning
    -------
    Some extensions (`Data Validation`) are not included
    into the exported file. This is a openpyxl limitation.

    """

    sheet_name = 'Timeline'
    header = 2

    def __init__(self, timeline: str | Path | None = None, itl: str | Path | None = None, **kwargs):
        self.fname = Path(timeline) if timeline else DEFAULT_TEMPLATE
        self._load_data()

        if itl:
            self.append(itl, **kwargs)

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.fname}')"

    def __len__(self) -> int:
        return len(self['OBS_NAME'])

    def __getitem__(self, name: str) -> list:
        return [
            col.value
            for col in self._timeline[self._fields[name]][self.header:]
            if col.value
        ]

    def __setitem__(self, item: (str, int), value: str | int | float) -> None:
        field, index = item
        self._timeline[self._fields[field] + str(index + self.header)] = value

    def _load_data(self) -> None:
        """Load file data."""
        # Disable openpyxl warning about unsupported extension (`Data validation`)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            self._template = load_workbook(self.fname, keep_vba=True)

        # Load Timeline spreadsheet
        self._timeline = self._template[self.sheet_name]

        # Load Timeline header fields mapping
        self._fields = {cell.value: cell.column_letter for cell in next(self._timeline.rows)}

    def append(
        self,
        itl: str | Path,
        ca_ref: str | dict | None = None,
        refs: dict | str | list | None = None
    ) -> None:
        """Append ITL observations blocks."""
        observations = read_itl(itl, refs=refs, flat=True)

        for i, obs in enumerate(observations, start=len(self['OBS_NAME']) + 1):
            for field, (key, fmt) in ITL_MAPPING.items():
                self[field, i] = fmt(obs[key])

            # Additional fields
            self['ITL name', i] = str(itl)  # ITL filename
            self['Mirror Flag', i] = 'ENABLE' if float(obs['START_SCAN_SPEED']) or float(obs['STOP_SCAN_SPEED']) else 'DISABLE'

            if ca_ref:
                self['First CU_frame start wrt C/A', i] = fmt_td(obs.start - datetime(dt_ca))
                self['Last CU_frame stop wrt C/A', i] = fmt_td(obs.stop - datetime(dt_ca))

    def save(self, fout: str | Path | None = None) -> Path:
        """Save MAJIS timeline."""
        if fout:
            fout = Path(fout)
        elif self.fname != DEFAULT_TEMPLATE:
            fout = self.fname
        else:
            raise FileExistsError('Can not overwrite default template. Please provide and export filename.')

        self._template.save(fout)

        return fout

    @property
    def log(self):
        return TimelineChangeLog(self._template)

    @property
    def version(self):
        """Template version."""
        return self.log[0][0]


timeline = Timeline()

timeline.append(*itl)

timeline['OBS_NAME', 2] = 'FOO'

timeline.append('../../tests/data/absolute_time.itl')

timeline.save('foo.xlsm')

timeline['OBS_NAME']

['MAJ_LEGA_SCAN_001', 'FOO', 'MAJ_LEGA_SCAN_001', 'MAJ_EGA_SCAN_001']

In [ ]:
timeline._fields

{'Event Name': 'A',
 'Phase': 'B',
 'block': 'C',
 'Comments': 'D',
 'OBS_NAME': 'E',
 'start_time_relative_ca': 'F',
 'stop_time_relative_ca': 'G',
 'start_time_utc': 'H',
 'stop_time_utc': 'I',
 'pointing desc': 'J',
 'MAJIS resol': 'K',
 'prime': 'L',
 'MAJIS ODF name': 'M',
 'ITL name': 'N',
 'Mirror Flag': 'O',
 'start_angle': 'P',
 'start_scan_speed': 'Q',
 'stop_scan_speed': 'R',
 'Scanner step per frame': 'S',
 'stop_angle': 'T',
 'scanner_timetot (ms)': 'U',
 'First CU_frame start wrt C/A': 'V',
 'Last CU_frame stop wrt C/A': 'W',
 'First CU_frame start (UTC)': 'X',
 'Last CU_frame stop (UTC)': 'Y',
 'cu_trep_ms': 'Z',
 'spatial_binning': 'AA',
 'obs durat (sec) (w/o borders)': 'AB',
 'nb_cu_frames_tot': 'AC',
 'nb_cu_frames_tot (computed)': 'AD',
 'ppe': 'AE',
 'Spectral Mapping': 'AF',
 'Number of spectels after spectral binning': 'AG',
 'Nb of bits per spectel after compression': 'AH',
 'datarate (bits/s)': 'AI',
 'DV (Mbits)': 'AJ',
 'Browse Parameters': 'AK',
 'Number of 

In [ ]:
timeline.version

'1.9.10'

In [ ]:
Timeline(itl='../../tests/data/absolute_time.itl').save('bar.xlsx')

In [5]:
timeline = template['Timeline']

timeline

<Worksheet "Timeline">

In [7]:
timeline['A1:B2']

((<Cell 'Timeline'.A1>, <Cell 'Timeline'.B1>),
 (<Cell 'Timeline'.A2>, <Cell 'Timeline'.B2>))

In [9]:
from majis.itl.timeline import Timeline

In [25]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm')['prime']

['MAJIS', 'MAJIS', 'MAJIS', 'MAJIS', 'MAJIS', 'MAJIS']

In [15]:
a = {'a': 1, 'b': 2}

a.values()[-1]

TypeError: 'dict_values' object is not subscriptable

In [31]:
timeline['A']

(<Cell 'Timeline'.A1>,
 <Cell 'Timeline'.A2>,
 <Cell 'Timeline'.A3>,
 <Cell 'Timeline'.A4>,
 <Cell 'Timeline'.A5>,
 <Cell 'Timeline'.A6>,
 <Cell 'Timeline'.A7>,
 <Cell 'Timeline'.A8>,
 <Cell 'Timeline'.A9>,
 <Cell 'Timeline'.A10>,
 <Cell 'Timeline'.A11>,
 <Cell 'Timeline'.A12>,
 <Cell 'Timeline'.A13>,
 <Cell 'Timeline'.A14>,
 <Cell 'Timeline'.A15>,
 <Cell 'Timeline'.A16>,
 <Cell 'Timeline'.A17>,
 <Cell 'Timeline'.A18>,
 <Cell 'Timeline'.A19>,
 <Cell 'Timeline'.A20>,
 <Cell 'Timeline'.A21>,
 <Cell 'Timeline'.A22>,
 <Cell 'Timeline'.A23>,
 <Cell 'Timeline'.A24>,
 <Cell 'Timeline'.A25>,
 <Cell 'Timeline'.A26>,
 <Cell 'Timeline'.A27>,
 <Cell 'Timeline'.A28>,
 <Cell 'Timeline'.A29>)

In [37]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm')[2]

{'Comments': 'NIGHT OBSERVATION OF KUALA LUMPUR AND SUMATRA ISLAND EAST COAST',
 'OBS_NAME': 'FOO',
 'prime': 'MAJIS',
 'start_angle': -1.99951,
 'start_scan_speed': 0.0235421317,
 'stop_scan_speed': 0.0235421317,
 'stop_angle': 2.09682,
 'First CU_frame start (UTC)': '2024-08-20T21:25',
 'Last CU_frame stop (UTC)': '2024-08-20T21:27:54',
 'cu_trep_ms': 200,
 'spatial_binning': 'No Binning',
 'nb_cu_frames_tot': 870,
 'ppe': 128,
 'Start Row VI': 372}

In [42]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm')['OBS_NAME', 2]

'MAJ_LEGA_SCAN_001'

In [38]:
timeline['1:2']

((<Cell 'Timeline'.A1>,
  <Cell 'Timeline'.B1>,
  <Cell 'Timeline'.C1>,
  <Cell 'Timeline'.D1>,
  <Cell 'Timeline'.E1>,
  <Cell 'Timeline'.F1>,
  <Cell 'Timeline'.G1>,
  <Cell 'Timeline'.H1>,
  <Cell 'Timeline'.I1>,
  <Cell 'Timeline'.J1>,
  <Cell 'Timeline'.K1>,
  <Cell 'Timeline'.L1>,
  <Cell 'Timeline'.M1>,
  <Cell 'Timeline'.N1>,
  <Cell 'Timeline'.O1>,
  <Cell 'Timeline'.P1>,
  <Cell 'Timeline'.Q1>,
  <Cell 'Timeline'.R1>,
  <Cell 'Timeline'.S1>,
  <Cell 'Timeline'.T1>,
  <Cell 'Timeline'.U1>,
  <Cell 'Timeline'.V1>,
  <Cell 'Timeline'.W1>,
  <Cell 'Timeline'.X1>,
  <Cell 'Timeline'.Y1>,
  <Cell 'Timeline'.Z1>,
  <Cell 'Timeline'.AA1>,
  <Cell 'Timeline'.AB1>,
  <Cell 'Timeline'.AC1>,
  <Cell 'Timeline'.AD1>,
  <Cell 'Timeline'.AE1>,
  <Cell 'Timeline'.AF1>,
  <Cell 'Timeline'.AG1>,
  <Cell 'Timeline'.AH1>,
  <Cell 'Timeline'.AI1>,
  <Cell 'Timeline'.AJ1>,
  <Cell 'Timeline'.AK1>,
  <Cell 'Timeline'.AL1>,
  <Cell 'Timeline'.AM1>,
  <Cell 'Timeline'.AN1>,
  <Cell 'Timeline'.AO1>,
 

In [41]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm')

Timeline('foo.xlsm')

In [44]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm').version

'1.9.10'

In [45]:
Timeline('../../tests/data/absolute_time.itl', timeline='foo.xlsm').log

Version,Date,Author,Changes
1.9.10,2024-05-29,Cydalise Dumesnil,"Add new column ""Comments"""
1.9.9,2024-05-23,Vincent Carlier,"Add new columns ""VI Gain"", ""VI Offset"", ""IR Gain"", ""IR Offset"""
1.9.8,2024-05-13,Vincent Carlier,"Fix issues related to the decimal separator ',' or '.' depending on the host PC : - new variable name 'decimal_separator' used to compute the time 'wrt C/A' from 'UTC' in the cells - fix of the macro DIFF_TIME_ms used to compute the DV in case of continuous acquisition Add a note about 'scanner_timetot' computation Add a note about decimal separator in the time 'wrt C/A' columns"
1.9.7,2024-04-25,Vincent Carlier,Add spectral masks
1.9.7,2024-03-21,Vincent Carlier,"Change the names BROWSE_TABLE into BROWSE_PARAMETERS Change column names: ""Browse"" into ""Browse Parameters"" Fix fomulas for nomi and brow to take into account VI_Flag and IR_Flag Add bullet tips in 2nd row (copy of footer comments)"
1.9.7,2024-03-14,Vincent Carlier,Split de number of spectels from the spectral table into VI and IR. Split de number of spectels from the browse parameters into VI and IR.
1.9.6,2024-03-14,Vincent Carlier,"Add a macro which resets the unused values if Mirror Flag is ""DISABLE"" Add a data validation on start_angle and stop_angle (from -2.1 to +2.1)"
1.9.6,2024-03-13,Vincent Carlier,- reorganisation of the order of columns
1.9.6,2024-03-11,Vincent Carlier,"- Change column names: ""Observation ID"" changed in ""OBS_NAME"""
1.9.6,2024-03-04,Vincent Carlier,- Add a check in DV cell to check that the number of CU Frame entered by hand is equal to the computed value - fix the DIFF_TIME_CA_ms function to work with column name instead of cell name - Update color code - Add comments


In [48]:
str(type(1.0).__name__)

'float'

In [51]:
[
    cell.value
    for cell in timeline['E']
]

['OBS_NAME',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [52]:
Timeline('../../tests/data/absolute_time.itl')

Timeline('/Users/seignovert-b/Documents/Research/Nantes/repos/ias/majis-ops-toolbox/src/majis/itl/timeline.xltm')